In [ ]:
from pathlib import Path
import pandas as pd
import plotly.express as px
from sklearn.metrics.pairwise import haversine_distances
import numpy as np
from datetime import datetime
import scipy.stats as scs
from statsmodels.distributions.copula.api import GumbelCopula, GaussianCopula
import matplotlib.pyplot as plt

# 1. Loading the data

In [ ]:
DATA_PATH = Path("data")
WEATHER_DATA_PATH = DATA_PATH / "RR59"

In [ ]:
weather_data = pd.read_csv(WEATHER_DATA_PATH / "Q_59_previous-1950-2022_RR-T-Vent.csv", sep = ";", parse_dates=["AAAAMMJJ"])

## 1.1 Selecting two stations of interest

In [ ]:
station_1 = "DUNKERQUE"
station_2 = "WATTEN"

joint_data = weather_data[weather_data["NOM_USUEL"].isin([station_1, station_2])]
joint_rainfall = joint_data.pivot_table(
    index="AAAAMMJJ", columns="NOM_USUEL", values="RR"
)

In [ ]:
m = px.scatter_mapbox(
    joint_data.drop_duplicates(subset=["LAT", "LON"]), lat="LAT", lon="LON"
)
m.update_layout(mapbox_style="carto-positron")

# 2. Time serie analysis

In [ ]:
joint_rainfall.plot(backend = "plotly")

In [ ]:
joint_rainfalls_corr = pd.DataFrame(
    [
        [
            joint_rainfall[station_1].shift(i).corr(joint_rainfall[station_2].shift(j))
            for i in range(0, 30)
        ]
        for j in range(0, 30)
    ]
)

In [ ]:
px.imshow(
    joint_rainfalls_corr,
    width=1800,
    height=1200,
    color_continuous_scale="Jet",
    range_color=(0, 1),
)

# 3. Study pairwise relationship

In [ ]:
threshold = 5

extreme_joint_rainfalls = joint_rainfall[(joint_rainfall > threshold).all(axis=1)].reset_index()

In [ ]:
extreme_joint_rainfalls.plot(
    backend="plotly",
    kind="scatter",
    x=station_1,
    y=station_2,
    trendline="ols",
    marginal_x="histogram",
    marginal_y="histogram",
    height=1200,
)

# 3 Fitting marginals

In [ ]:
var_range = np.linspace(threshold*2,100)

In [ ]:
gompertz_fit_x = scs.gompertz.fit(extreme_joint_rainfalls[station_1])
print(gompertz_fit_x)

hist = extreme_joint_rainfalls[station_1].hist(
    backend="plotly", log_x=True, opacity=0.5, histnorm="probability"
)
gompertz_plot = px.line(x=var_range, y=scs.gompertz.pdf(var_range, *gompertz_fit_x))
hist.add_trace(gompertz_plot.data[0])

In [ ]:
gompertz_fit_y = scs.gompertz.fit(extreme_joint_rainfalls[station_2])
print(gompertz_fit_y)

hist = extreme_joint_rainfalls[station_2].hist(
    backend="plotly", log_x=True, opacity=0.5, histnorm="probability"
)
gompertz_plot = px.line(x=var_range, y=scs.gompertz.pdf(var_range, *gompertz_fit_y))
hist.add_trace(gompertz_plot.data[0])

# 4. Pairwise relationship

In [ ]:
x_ppf = scs.gompertz.cdf(extreme_joint_rainfalls[station_1], *gompertz_fit_x)
y_ppf = scs.gompertz.cdf(extreme_joint_rainfalls[station_2], *gompertz_fit_y)

In [ ]:
plt.scatter(x_ppf, y_ppf, s=1)

In [ ]:
plt.hexbin(x=x_ppf, y=y_ppf, gridsize=30)

In [ ]:
corr = GaussianCopula().fit_corr_param(
    data=extreme_joint_rainfalls[[station_1, station_2]]
)
gaussian_copula = GaussianCopula(corr=corr)
_ = gaussian_copula.plot_pdf()

In [ ]:
corr = GumbelCopula().fit_corr_param(
    data=extreme_joint_rainfalls[[station_1, station_2]]
)
gumbel_copula = GumbelCopula(theta=corr)
_ = gumbel_copula.plot_pdf()

# 5. Sample data

In [ ]:
N = int(1e6)

In [ ]:
u = gumbel_copula.rvs(N)

In [ ]:
plt.hexbin(u[:, 0], u[:, 1], gridsize = 25)

In [ ]:
plt.hist(u[:, 0])

In [ ]:
sample_x = scs.gompertz.ppf(u[:, 0], *gompertz_fit_x)
sample_y = scs.gompertz.ppf(u[:, 1], *gompertz_fit_y)

In [ ]:
plt.scatter(sample_x, sample_y, s = 0.2)

What is the return period of an event trigerring both stations at 40mm rainfall ?

In [ ]:
1/((sample_x > 50) & (sample_y > 50)).mean()